# Generalised Volatility
## The generalised volativity of a portfolio is:
$\sigma_p ^2 = \sum_{i=1}^k \sum_{j=1}^k \omega_i \omega_j \sigma_i \sigma_j \rho_{ij} = \omega^T \Sigma \omega
$

#### where $\rho$ is the correlation matrix and $\sigma_i \sigma_j \rho_{ij}$ is the covariance


<img src="images\Efficient frontier 1.PNG" width="400px">
We know there exist asset with highest and lowest possible returns, these will be the end points of the Efficient Frontier. We can then split between R_max and R_min into many steps, and find the least possible volativity for each R between the min and max. This way, we can numerically calculate the efficient frontier.

### Minimise: $\frac{1}{2}w^T\Sigma w$

### Subject to the constraints:

$w^T R=r_o$

$w^T1=1$

$w\geq0$

In [1]:
import pandas as pd
import numpy as np
import scipy.stats
import edhec_risk_kit as erk 
%load_ext autoreload
%autoreload 2

In [21]:
def portfolio_return(weights, returns):
    """
    Weights -> Returns
    """
    return weights.T @ returns

def portfolio_vol(weights, cov):
    """
    Weifghts -> Vol
    """
    return (weights.T @ cov @weights) ** 0.5

# N-Asset Efficient Frontier

In [25]:
ind = erk.get_ind_returns(weighting='vw', n_inds=30)
er = erk.ann_return(ind["1996":"2000"])
cov = ind["1996":"2000"].cov()

In [23]:
def minimize_vol(target_return, er, cov):
    """
    target_ret -> W
    """
    n = er.shape[0]
    init_guess = np.repeat(1/n, n)
    bounds = ((0.0, 1),)*n
    return_is_target = {
        "type": "eq",
        "args": (er,),
        "fun": lambda weights, er: target_return - erk.portfolio_return(weights, er)
    }
    weights_sum_to_1 = {
            "type": "eq",
            "fun": lambda weights: np.sum(weights) - 1
    }
    results = minimize(erk.portfolio_vol, init_guess,
                      args=(cov,), method="SLSQP",
                       options={"disp": False},
                       constraints=(return_is_target, weights_sum_to_1),
                       bounds=bounds
                      )
    return results.x


In [26]:
def optimal_weights(n_points, er, cov):
    """
    -> list of weights to run the optimizer on to minimize the vol
    """
    target_returns = np.linspace(er.min(),er.max(),n_points)
    weights = [minimize_vol(target_return, er, cov) for target_return in target_returns]
    return weights

def plot_ef(n_points, er, cov, style=".-"):
    """
    Plots the N-asset efficient frontier
    """
    weights = optimal_weights(n_points, er, cov)
    rets = [portfolio_return(w, er) for w in weights]
    vol = [portfolio_vol(w, cov) for w in weights]
    ef = pd.DataFrame({
        "Returns": rets,
        "Volativity": vol
    })
    return ef.plot.line(x="Volativity", y="Returns", style=style)

In [27]:
l=["Smoke", "Fin", "Games", "Coal"]
erk.plot_ef(50, er, cov)

<Figure size 432x288 with 1 Axes>

# Max Sharpe Ratio Portfolio

<img src="images\Max Sharpe Ratio Portfolio.PNG">

Introducing risk free asset give rise to very different efficient frontier. Any straight line from the risk free rate asset and the riskful portfolio can be a new portfolio.
Therefore we would like to find the portfolio that gives the highest return per unit risk.
### $R_p = (1-w)R_{rf} + w R$
### $\sigma_p = w \sigma$
### we get:
### $Max \frac{dR_p}{d\sigma_p} = Max \frac{R -R_{rf}}{\sigma}$
Therefore maximising the gradient of the straight line intersecting risk-free rate and the risk-ful portfolio gives the Max Sharpe Ratio Portfolio
It can also be shown that a maximum sharpe ratio portfolio contains no specific risks, only systematic risks

In [119]:
import pandas as pd
import numpy as np
import scipy.stats
import matplotlib.pyplot as plt
import edhec_risk_kit as erk 
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [112]:
def portfolio_return(weights, returns):
    """
    Weights -> Returns
    """
    return weights.T @ returns

def portfolio_vol(weights, cov):
    """
    Weifghts -> Vol
    """
    return (weights.T @ cov @weights) ** 0.5

In [113]:
from scipy.optimize import minimize
def maximize_sharpe_ratio(riskfree_rate, er, cov):
    """
    target_ret -> W
    """
    n = er.shape[0]
    init_guess = np.repeat(1/n, n)
    bounds = ((0.0, 1),)*n

    weights_sum_to_1 = {
            "type": "eq",
            "fun": lambda weights: np.sum(weights) - 1
    }
    
    def neg_sharpe_ratio(weights, er, cov):
        return -(portfolio_return(weights, er)-riskfree_rate)/portfolio_vol(weights, cov)
    
    results = minimize(neg_sharpe_ratio, init_guess,
                      args=(er, cov), method="SLSQP",
                       options={"disp": False},
                       constraints=(weights_sum_to_1),
                       bounds=bounds
                      )
    return results.x

In [28]:
ind = erk.get_ind_returns(weighting='vw', n_inds=30)
er = erk.ann_return(ind["1996":"2000"])
cov = ind["1996":"2000"].cov()

erk.plot_ef(n_points=20, er=er, cov=cov, show_cml=True, riskfree_rate=0.1)

<Figure size 432x288 with 1 Axes>

# Lack of robustness of Markowitz analysis, and the GMV
- Estimation error is the key challenge in portfolio estimization. If you feed an optimizer data with estimation error, this error will be amplified by the optimizer. 
- The assets that get the most allocation will typically the ones that have the highest estimation errors. 
- Expected return estimates is much harder to obtain accurately compared to the covariance matrix. The sample-based expected return parameter estimate are very noisy. We are very bad at guessing expected returns! 
- Thus in practice, we often focus on the only portfolio on the efficient frontier in which no return parameter is needed. 
- One example is the Global Minimum Variance (GMV) portfolio. Which is the portfolio that minimize variance without an expected return target.

In [130]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import edhec_risk_kit as erk
import numpy as np
import pandas as pd

ind = erk.get_ind_returns(weighting='vw', n_inds=30)
er = erk.ann_return(ind["1996":"2000"])
cov = ind["1996":"2000"].cov()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [122]:
l = ["Food", "Steel"]
erk.maximize_sharpe_ratio(0.1, er[l], cov.loc[l,l])

array([0.75040362, 0.24959638])

Assume returns that slightly deviates from historical data:

- Even deviation of 0.02 can give vastly different portfolio!
- This is whats called "Error maximizing nature of Markowitz"

In [123]:
erk.maximize_sharpe_ratio(0.1, np.array([0.11, 0.12]), cov.loc[l,l])

array([0.57930354, 0.42069646])

This is a very different portfolio!
- Before: 0.75 in food, 0.25 in steel
- Now: 0.57 in food, 0.42 in steel!

In [124]:
erk.maximize_sharpe_ratio(0.1, np.array([0.10, 0.13]), cov.loc[l,l])

array([0., 1.])

100% in steel!!!

In [125]:
erk.maximize_sharpe_ratio(0.1, np.array([0.13, 0.10]), cov.loc[l,l])

array([1., 0.])

100% in food!!

# MSR, GMV, EW portfolios on the Efficient Frontier

In [132]:
def plot_ef(n_points, er, cov, show_cml=False, style=".-", riskfree_rate=0, show_ew=False, show_GMV=False):
    """
    Plots the N-asset efficient frontier
    """
    weights = optimal_weights(n_points, er, cov)
    rets = [portfolio_return(w, er) for w in weights]
    vol = [portfolio_vol(w, cov) for w in weights]
    ef = pd.DataFrame({
        "Returns": rets,
        "Volativity": vol
    })
    
    ax = ef.plot.line(x="Volativity", y="Returns", style=style)
    
    if show_cml:
        ax.set_xlim(left=0)
        weight_msr = maximize_sharpe_ratio(riskfree_rate, er, cov)
        r_msr = portfolio_return(weight_msr, er)
        vol_msr = portfolio_vol(weight_msr, cov)
        # Add Capital Market Line
        cml_x = [0, vol_msr]
        cml_y = [riskfree_rate, r_msr]
        ax.plot(cml_x, cml_y, color="green", marker="o", linestyle="dashed", markersize=12, linewidth=2)
        
    if show_ew:
        n = er.shape[0]
        w_ew = np.repeat(1/n, n)
        r_ew = portfolio_return(w_ew, er)
        vol_ew = portfolio_vol(w_ew, cov)
        ax.plot([vol_ew], [r_ew], color="goldenrod", marker="o", markersize=12)

        
    if show_GMV:
        
        w_GMV = GMV(cov)
        r_GMV = portfolio_return(w_GMV, er)
        vol_GMB = portfolio_vol(w_GMV, cov)
        ax.plot([vol_GMB],[r_GMV], color="red", marker="o", markersize=12)
             
    return ax

In [29]:
erk.plot_ef(n_points=20, er=er, cov=cov, show_cml=True, riskfree_rate=0.1, show_ew=True, show_GMV=True)

<Figure size 432x288 with 1 Axes>